In [2]:
import geopandas as gpd

# Charger les itinéraires (fichier contenant les polylignes OD et la demande associée à chaque polyligne)
gdf_routes = gpd.read_file("output/itineraire_global_TIM.shp")

# Charger les périmètres des quartiers
gdf_corridors = gpd.read_file("input/corridors.gpkg")
gdf_quartiers = gpd.read_file("input/Quartiers_ZonesMMT.shp")

In [3]:
#Homogénéisation des coordonnées
print(gdf_routes.crs, gdf_quartiers.crs, gdf_corridors.crs) #verif des coordonnées
gdf_quartiers = gdf_quartiers.to_crs(gdf_routes.crs)
gdf_corridors = gdf_corridors.to_crs(gdf_routes.crs)
print(gdf_routes.crs, gdf_quartiers.crs, gdf_corridors.crs) #verif des coordonnées

EPSG:4326 EPSG:2056 EPSG:2056
EPSG:4326 EPSG:4326 EPSG:4326


In [4]:
gdf_quartiers["demande_transit"] = 0
gdf_quartiers["demande_entree"] = 0
gdf_quartiers["demande_sortie"] = 0
gdf_corridors["demande_transit"] = 0
gdf_corridors["demande_entree"] = 0
gdf_corridors["demande_sortie"] = 0

for index, quartier in gdf_quartiers.iterrows():
    nom_quartier = quartier["Quartiers"]  
    geom_quartier = quartier.geometry  

    itinéraires_transit = gdf_routes[
        (gdf_routes.geometry.intersects(geom_quartier)) &
        (gdf_routes["start_name"] != nom_quartier) &
        (gdf_routes["end_name"] != nom_quartier)
    ]
    itinéraires_entree = gdf_routes[(gdf_routes["end_name"] == nom_quartier)]
    itinéraires_sortie = gdf_routes[(gdf_routes["start_name"] == nom_quartier)]

    # Somme des demandes pour ces itinéraires en transit, en entrée ou en sortie 
    gdf_quartiers.at[index, "demande_transit"] = itinéraires_transit["demande"].sum()
    gdf_quartiers.at[index, "demande_entree"] = itinéraires_entree["demande"].sum()
    gdf_quartiers.at[index, "demande_sortie"] = itinéraires_sortie["demande"].sum()
    
for index, quartier in gdf_corridors.iterrows():
    nom_quartier = quartier["Nom_corridor"]  
    geom_quartier = quartier.geometry  

    itinéraires_transit = gdf_routes[
        (gdf_routes.geometry.intersects(geom_quartier)) &
        (gdf_routes["start_name"] != nom_quartier) &
        (gdf_routes["end_name"] != nom_quartier)
    ]
    itinéraires_entree = gdf_routes[(gdf_routes["end_name"] == nom_quartier)]
    itinéraires_sortie = gdf_routes[(gdf_routes["start_name"] == nom_quartier)]

    # Somme des demandes pour ces itinéraires
    gdf_corridors.at[index, "demande_transit"] = itinéraires_transit["demande"].sum()
    gdf_corridors.at[index, "demande_entree"] = itinéraires_entree["demande"].sum()
    gdf_corridors.at[index, "demande_sortie"] = itinéraires_sortie["demande"].sum()
    
print(gdf_quartiers[["Quartiers", "demande_transit", "demande_entree", "demande_sortie"]].head())

C:\Users\gpe\AppData\Local\Temp\ipykernel_23968\557382738.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '41345.591775' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf_quartiers.at[index, "demande_transit"] = itinéraires_transit["demande"].sum()
C:\Users\gpe\AppData\Local\Temp\ipykernel_23968\557382738.py:22: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1990.507872' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf_quartiers.at[index, "demande_entree"] = itinéraires_entree["demande"].sum()
C:\Users\gpe\AppData\Local\Temp\ipykernel_23968\557382738.py:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '3603.171458' has dtype incompatible with int64, ple

        Quartiers  demande_transit  demande_entree  demande_sortie
0             AIG     41345.591775     1990.507872     3603.171458
1   Aire-la-Ville      1042.045913      568.289134      479.441490
2  Aïre-Le Lignon     12279.630995     3401.809568     3102.046847
3         Anières      6784.803370      877.705895      691.820514
4     Aubert-Rieu     13980.575272     1848.419410     2129.551766


In [5]:
gdf_quartiers.to_excel("output/transit_TIM_quartiers.xlsx")
gdf_quartiers.to_file("output/transit_TIM_quartiers.gpkg")
gdf_corridors.to_excel("output/transit_TIM_corridor.xlsx")
gdf_corridors.to_file("output/transit_TIM_corridors.gpkg")

C:\Users\gpe\AppData\Local\Temp\ipykernel_23968\2939957097.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  gdf_quartiers.to_excel("output/transit_TIM_quartiers.xlsx")
C:\Users\gpe\AppData\Local\Temp\ipykernel_23968\2939957097.py:3: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.3' currently installed).
  gdf_corridors.to_excel("output/transit_TIM_corridor.xlsx")
